In [1]:
import numpy as np
import pandas as pd

 Reading in the vcf file for chromosome 1
 Skipping the first 6 rows since they are not in tabular format

In [ ]:
df = pd.read_csv("/opt/hpc4ag/data/3k-core-v7-chr1/chr1.vcf", sep="\t", skiprows=6)

In [20]:
df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,B001_B001,...,IRIS_313-15901_IRIS_313-15901,IRIS_313-15902_IRIS_313-15902,IRIS_313-15903_IRIS_313-15903,IRIS_313-15904_IRIS_313-15904,IRIS_313-15905_IRIS_313-15905,IRIS_313-15906_IRIS_313-15906,IRIS_313-15907_IRIS_313-15907,IRIS_313-15908_IRIS_313-15908,IRIS_313-15909_IRIS_313-15909,IRIS_313-15910_IRIS_313-15910
0,1,1178,1178,G,T,.,.,PR,GT,0/0,...,./.,./.,0/0,0/0,0/0,0/0,./.,1/1,1/1,0/0
1,1,1203,1203,T,C,.,.,PR,GT,0/0,...,./.,0/0,1/1,./.,1/1,0/0,0/0,1/1,1/1,0/0
2,1,1248,1248,G,A,.,.,PR,GT,0/0,...,0/0,0/0,0/0,0/0,1/1,0/0,0/0,0/0,0/0,0/0
3,1,1282,1282,G,A,.,.,PR,GT,0/0,...,0/0,0/0,./.,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,1,1299,1299,T,C,.,.,PR,GT,0/0,...,0/0,0/0,./.,0/0,0/0,0/0,0/0,1/1,1/1,0/0


In [23]:
df.shape

(42466, 3033)

Reading in the additive matrix file generated using plink
This is a space delimited file
Here, the rows indicate individuals genotyped, and columns are the loci in addition to some 
initial columns identifying if it was a mother or father in a cross or a sex chromosome or if there is an associated phenotype 


In [ ]:
df2 = pd.read_csv("/opt/hpc4ag/data/3k-core-v7-chr1/chr1_A.raw", sep='\\s+')

In [53]:
df2.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,1178_T,1203_C,1248_A,1282_A,...,43243718_G,43244177_T,43249269_T,43250351_G,43250373_A,43250392_T,43250427_A,43253620_T,43254222_T,43255617_A
0,B001,B001,0,0,0,-9,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B002,B002,0,0,0,-9,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,B003,B003,0,0,0,-9,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,B004,B004,0,0,0,-9,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B005,B005,0,0,0,-9,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df2.shape

(3024, 42472)

In [55]:
df2.tail()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,1178_T,1203_C,1248_A,1282_A,...,43243718_G,43244177_T,43249269_T,43250351_G,43250373_A,43250392_T,43250427_A,43253620_T,43254222_T,43255617_A
3019,IRIS_313-15906,IRIS_313-15906,0,0,0,-9,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3020,IRIS_313-15907,IRIS_313-15907,0,0,0,-9,NaN,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0
3021,IRIS_313-15908,IRIS_313-15908,0,0,0,-9,2.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3022,IRIS_313-15909,IRIS_313-15909,0,0,0,-9,2.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3023,IRIS_313-15910,IRIS_313-15910,0,0,0,-9,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0


Doing some exploratory analysis on some of the columns

In [56]:
df2["FID"].nunique()

3024

In [57]:
df2["PAT"].unique().tolist()

[0]

In [58]:
df2["MAT"].unique().tolist()

[0]

In [59]:
df2["SEX"].unique().tolist()

[0]

In [60]:
df2["PHENOTYPE"].unique().tolist()

[-9]

Seeing as columns MAT PAT SEX and PHENOTYPE have no data of interest as such, we can drop them before proceeding

In [62]:
labels = ["FID", "PAT", "MAT", "SEX", "PHENOTYPE"]
df2 = df2.drop(columns=labels, inplace = False)

In [63]:
df2.head()

,IID,1178_T,1203_C,1248_A,1282_A,1299_C,1306_A,1792_A,5752_T,6634_A,...,43243718_G,43244177_T,43249269_T,43250351_G,43250373_A,43250392_T,43250427_A,43253620_T,43254222_T,43255617_A
0,B001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,B003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,B004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
df2.shape

(3024, 42467)

We can make the individual ID column into an index so it becomes easier to transpose the matrix if required

In [66]:
df2 = df2.set_index('IID')

In [67]:
df2.head()

,1178_T,1203_C,1248_A,1282_A,1299_C,1306_A,1792_A,5752_T,6634_A,11334_C,...,43243718_G,43244177_T,43249269_T,43250351_G,43250373_A,43250392_T,43250427_A,43253620_T,43254222_T,43255617_A
IID,,,,,,,,,,,,,,,,,,,,,
B001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
B004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df2.head()

,1178_T,1203_C,1248_A,1282_A,1299_C,1306_A,1792_A,5752_T,6634_A,11334_C,...,43243718_G,43244177_T,43249269_T,43250351_G,43250373_A,43250392_T,43250427_A,43253620_T,43254222_T,43255617_A
IID,,,,,,,,,,,,,,,,,,,,,
B001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
B004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
df2.isnull().any(axis=1).sum()

3024

Because a large number of missing values are present, we need to impute the values instead of simply dropping those rows or loci

In [76]:
cols = df2.columns.tolist()
df2[cols] = df2[cols].fillna(df2.mode().iloc[0])

In [75]:
df2.dtypes

1178_T        float64
1203_C        float64
1248_A        float64
1282_A        float64
1299_C        float64
               ...   
43250392_T    float64
43250427_A    float64
43253620_T    float64
43254222_T    float64
43255617_A    float64
Length: 42466, dtype: object

In [77]:
df2.head()

,1178_T,1203_C,1248_A,1282_A,1299_C,1306_A,1792_A,5752_T,6634_A,11334_C,...,43243718_G,43244177_T,43249269_T,43250351_G,43250373_A,43250392_T,43250427_A,43253620_T,43254222_T,43255617_A
IID,,,,,,,,,,,,,,,,,,,,,
B001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
B004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
df2.tail()

,1178_T,1203_C,1248_A,1282_A,1299_C,1306_A,1792_A,5752_T,6634_A,11334_C,...,43243718_G,43244177_T,43249269_T,43250351_G,43250373_A,43250392_T,43250427_A,43253620_T,43254222_T,43255617_A
IID,,,,,,,,,,,,,,,,,,,,,
IRIS_313-15906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_313-15907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0
IRIS_313-15908,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
IRIS_313-15909,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_313-15910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0


In [79]:
df2["1178_T"].value_counts()

1178_T
0.0    2831
2.0     179
1.0      14
Name: count, dtype: int64

In [80]:
# Transposing the matrix so we have the loci as the rows and individuals as the columns
df2 = df2.T

In [81]:
df2.head()

IID,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,IRIS_313-15901,IRIS_313-15902,IRIS_313-15903,IRIS_313-15904,IRIS_313-15905,IRIS_313-15906,IRIS_313-15907,IRIS_313-15908,IRIS_313-15909,IRIS_313-15910
1178_T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0
1203_C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0
1248_A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1282_A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1299_C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0


In [82]:
# Counting the number of rows that have null values
df2.shape[0] - df2.dropna().shape[0]

0

In [83]:
df2.shape

(42466, 3024)

In [84]:
# Saving this as a csv file for further processing
df2.to_csv("chr1_A_matrix.csv")